## Arkouda example of cosine distance and euclidean distance
- Two random arkouda int64 pdarrays are created then the distance is measured between them...
- The cosine and euclidean distance functions are compared against the scipy variants for correctness

Arkouda functions used:
- `ak.connect`
- `ak.randint`
- `ak.sum`
- `ak.pdarray.__mul__`
- `ak.pdarray.to_ndarray`
- `ak.disconnect`
- `ak.shutdown`

In [ ]:
import arkouda as ak

In [ ]:
import math
from scipy.spatial import distance
import numpy as np

### Connect to the Arkouda server

In [ ]:
# connect to the arkopuda server using the connect_url which the server prints out
ak.connect(connect_url="tcp://localhost:5555")

### Create two pdarrays and fill with random integers

In [ ]:
# create two in64 pdarrays and add them
a = ak.randint(0,10,100)
b = ak.randint(0,10,100)

print(a+b)

### Check for pdarray

In [ ]:
def ak_check_pda(u):
    if not isinstance(u, ak.pdarray):
        raise TypeError("argument must be a pdarray")

### Dot Product of two Arkouda pdarrays
$u \cdot v = \sum_i{u_i v_i}$

In [ ]:
# define the dot product or two arkouda pdarrays
def ak_dot(u, v):
    ak_check_pda(u)
    ak_check_pda(v)
    return ak.sum(u*v)

### Magnatude ($L_2$ norm) of an Arkouda pdarray
$\|u\|_2 = \sqrt{\sum_i{u_i^2}}$

In [ ]:
# define the magnitude/L_2-norm of arkouda pdarray
def ak_mag2(u):
    ak_check_pda(u)
    return math.sqrt(ak_dot(u,u))

### Cosine Distance of two Arkouda pdarrays
$D_C = 1 - \cos(\theta) = 1 - \frac{u \cdot v}{\|u\|_2\|v\|_2} = 1 - \frac{\sum_i{u_i v_i}}{\sqrt{\sum_i{u_i^2}}\sqrt{\sum_i{v_i^2}}}$

In [ ]:
# define the cosine distance of two arkouda pdarrays
# should function similarly to scipy.spatial.distance.cosine
def ak_cos_dist(u, v):
    ak_check_pda(u)
    ak_check_pda(v)
    return (1.0 - ak_dot(u,v)/(ak_mag2(u)*ak_mag2(v)))

### Euclidean Distance of two Arkouda pdarrays
$D_E = \|u-v\|_2$

In [ ]:
# define the euclidean distance of two arkouda pdarrays
# should function similarly to scipy.spatial.distance.euclidean
def ak_euc_dist(u, v):
    ak_check_pda(u)
    ak_check_pda(v)
    return (ak_mag2(u-v))

### Measure cosine distance and check against scipy

In [ ]:
# check the arkouda version against the scipy version
d1 = ak_cos_dist(a,b)
d2 = distance.cosine(a.to_ndarray(), b.to_ndarray())
print(d1,d2)
print(np.allclose(d1,d2))

### Measure euclidean distance and check against scipy

In [ ]:
# check the arkouda version against the scipy version
d1 = ak_euc_dist(a,b)
d2 = distance.euclidean(a.to_ndarray(), b.to_ndarray())
print(d1,d2)
print(np.allclose(d1,d2))

### Disconnect from Arkouda server or Shutdown Arkouda server

In [ ]:
# disconnect from the the arkouda server
ak.disconnect()
# shutdown the arkouda server
#ak.shutdown()